In [2]:
import numpy as np
import pandas as pd
import datetime
import math

In [17]:
dam = pd.read_csv('dam.csv')
dam

,hourlyid,market,tradedate,hour,price,volume
0,26345,1,2016-01-01,0,40.10,477.5
1,26346,1,2016-01-01,1,40.08,486.6
2,26347,1,2016-01-01,2,40.06,495.6
3,26348,1,2016-01-01,3,40.08,496.6
4,26349,1,2016-01-01,4,40.07,502.9
...,...,...,...,...,...,...
46723,73029,1,2021-04-29,4,39.97,126.1
46724,73037,1,2021-04-29,12,68.06,71.0
46725,73049,1,2021-04-30,0,44.04,59.6
46726,73057,1,2021-04-30,8,190.00,72.0


In [18]:
dam['date']=pd.to_datetime(dam['tradedate'],format="%Y/%m/%d")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100

dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
dam=dam.drop(['hourlyid','tradedate','date'],axis=1)
dam

,market,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,
2016-01-01,1,0,40.10,477.5,2016,1,1,4,2016.01,weekday_0
2016-01-01,1,1,40.08,486.6,2016,1,1,4,2016.01,weekday_1
2016-01-01,1,2,40.06,495.6,2016,1,1,4,2016.01,weekday_2
2016-01-01,1,3,40.08,496.6,2016,1,1,4,2016.01,weekday_3
2016-01-01,1,4,40.07,502.9,2016,1,1,4,2016.01,weekday_4
...,...,...,...,...,...,...,...,...,...,...
2021-04-29,1,4,39.97,126.1,2021,4,29,3,2021.04,weekday_4
2021-04-29,1,12,68.06,71.0,2021,4,29,3,2021.04,weekday_12
2021-04-30,1,0,44.04,59.6,2021,4,30,4,2021.04,weekday_0


In [20]:
dam['price_vol']=dam['price']*dam['volume']
daily=dam.groupby(['year','month','day']).sum().drop(['hour','day_of_week','month_no','market'],axis=1)
daily['price_weight_av']=daily['price_vol']/daily['volume']
daily=daily.drop(['price','price_vol'], axis=1)
daily['price_max']=dam[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=dam[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=dam[['year','month','day','price']].groupby(['year','month','day']).mean()
daily

volume  price_weight_av  price_max  price_min  price_mean
year month day                                                           
2016 1     1    7873.8        42.785251      60.08      35.09   45.982500
           2    7319.5        47.268532     118.49      40.03   54.114167
           3    7198.7        48.857812     118.47      44.06   54.959167
           4    4457.0        50.693024     177.66      46.70   80.164583
           5    3750.4        47.746609     181.40      42.06   84.503333
...                ...              ...        ...        ...         ...
2021 4     26   2092.9        88.683281     299.33      39.94  102.215000
           27   2048.2        82.150912     299.09      39.96   94.468333
           28   2130.2        83.346161     299.10      39.90  100.343333
           29   2134.0        83.738512     296.82      39.79   98.742500
           30   2198.9        85.068659     295.98      38.97   98.218750

[1947 rows x 5 columns]

In [21]:
dam=dam.reset_index().set_index(['year','month','day'])
dam

date  market  hour   price  volume  day_of_week  \
year month day                                                         
2016 1     1   2016-01-01       1     0   40.10   477.5            4   
           1   2016-01-01       1     1   40.08   486.6            4   
           1   2016-01-01       1     2   40.06   495.6            4   
           1   2016-01-01       1     3   40.08   496.6            4   
           1   2016-01-01       1     4   40.07   502.9            4   
...                   ...     ...   ...     ...     ...          ...   
2021 4     29  2021-04-29       1     4   39.97   126.1            3   
           29  2021-04-29       1    12   68.06    71.0            3   
           30  2021-04-30       1     0   44.04    59.6            4   
           30  2021-04-30       1     8  190.00    72.0            4   
           30  2021-04-30       1    17   67.97   114.2            4   

                month_no    hour_ref  price_vol  
year month day                                   
2016 1     1     2016.01   weekday_0  19147.750  
           1     2016.01   weekday_1  19502.928  
           1     2016.01   weekday_2  19853.736  
           1     2016.01   weekday_3  19903.728  
           1     2016.01   weekday_4  20151.203  
...                  ...         ...        ...  
2021 4     29    2021.04   weekday_4   5040.217  
           29    2021.04  weekday_12   4832.260  
           30    2021.04   weekday_0   2624.784  
           30    2021.04   weekday_8  13680.000  
           30    2021.04  weekday_17   7762.174  

[46728 rows x 9 columns]

In [57]:
daily['center']=dam.loc[dam['hour']==14,'price']
daily['base']=dam.loc[dam['hour']==1,'price']
daily['boundary1']=daily['base']+((daily['center']-daily['base'])/2)
daily['boundary2']=daily['center']+((daily['center']-daily['base'])/2)

daily

volume  price_weight_av  price_max  price_min  price_mean  \
year month day                                                              
2016 1     1    7873.8        42.785251      60.08      35.09   45.982500   
           2    7319.5        47.268532     118.49      40.03   54.114167   
           3    7198.7        48.857812     118.47      44.06   54.959167   
           4    4457.0        50.693024     177.66      46.70   80.164583   
           5    3750.4        47.746609     181.40      42.06   84.503333   
...                ...              ...        ...        ...         ...   
2021 4     26   2092.9        88.683281     299.33      39.94  102.215000   
           27   2048.2        82.150912     299.09      39.96   94.468333   
           28   2130.2        83.346161     299.10      39.90  100.343333   
           29   2134.0        83.738512     296.82      39.79   98.742500   
           30   2198.9        85.068659     295.98      38.97   98.218750   

                boundary1  boundary2  off_price_mean  standard_price_mean  \
year month day                                                              
2016 1     1       42.585     47.595       39.081000            45.095000   
           2       47.055     53.565       43.175000            50.717143   
           3       47.235     52.825       44.722857            50.334545   
           4       63.430     96.610       49.483000            80.024000   
           5       61.095     99.005       43.756667            80.042500   
...                   ...        ...             ...                  ...   
2021 4     26      55.000     85.100       42.868889            71.281250   
           27      54.990     85.050       42.197778            70.017500   
           28      54.970     85.050       42.605556            70.012500   
           29      53.930     82.190       42.532222            69.275556   
           30      53.480     82.480       42.047778            67.981250   

                peak_price_mean  off_hour_count  standard_hour_count  \
year month day                                                         
2016 1     1          53.239000            10.0                  4.0   
           2          73.138571            10.0                  7.0   
           3          75.380000             7.0                 11.0   
           4         157.220000            10.0                 10.0   
           5         141.990000             9.0                  8.0   
...                         ...             ...                  ...   
2021 4     26        213.870000             9.0                  8.0   
           27        189.617143             9.0                  8.0   
           28        209.241429             9.0                  8.0   
           29        227.258333             9.0                  9.0   
           30        204.995714             9.0                  8.0   

                peak_hour_count  center   base  
year month day                                  
2016 1     1               10.0   45.09  40.08  
           2                7.0   50.31  43.80  
           3                6.0   50.03  44.44  
           4                4.0   80.02  46.84  
           5                7.0   80.05  42.14  
...                         ...     ...    ...  
2021 4     26               7.0   70.05  39.95  
           27               7.0   70.02  39.96  
           28               7.0   70.01  39.93  
           29               6.0   68.06  39.80  
           30               7.0   67.98  38.98  

[1947 rows x 15 columns]

In [58]:
dam['boundary1']=daily['boundary1']
dam['boundary2']=daily['boundary2']


dam.loc[dam['price'] < dam['boundary1'],'cat']='off'
dam.loc[(dam['price'] >= dam['boundary1']) & (dam['price'] <= dam['boundary2']),'cat']='standard'
dam.loc[dam['price'] > dam['boundary2'],'cat']='peak'

dam.loc[dam['boundary2'] - dam['boundary1']<50,'cat']='off'
dam

date  market  hour   price  volume  day_of_week  \
year month day                                                         
2016 1     1   2016-01-01       1     0   40.10   477.5            4   
           1   2016-01-01       1     1   40.08   486.6            4   
           1   2016-01-01       1     2   40.06   495.6            4   
           1   2016-01-01       1     3   40.08   496.6            4   
           1   2016-01-01       1     4   40.07   502.9            4   
...                   ...     ...   ...     ...     ...          ...   
2021 4     29  2021-04-29       1     4   39.97   126.1            3   
           29  2021-04-29       1    12   68.06    71.0            3   
           30  2021-04-30       1     0   44.04    59.6            4   
           30  2021-04-30       1     8  190.00    72.0            4   
           30  2021-04-30       1    17   67.97   114.2            4   

                month_no    hour_ref  price_vol  boundary1  boundary2  cat  
year month day                                                              
2016 1     1     2016.01   weekday_0  19147.750     42.585     47.595  off  
           1     2016.01   weekday_1  19502.928     42.585     47.595  off  
           1     2016.01   weekday_2  19853.736     42.585     47.595  off  
           1     2016.01   weekday_3  19903.728     42.585     47.595  off  
           1     2016.01   weekday_4  20151.203     42.585     47.595  off  
...                  ...         ...        ...        ...        ...  ...  
2021 4     29    2021.04   weekday_4   5040.217     53.930     82.190  off  
           29    2021.04  weekday_12   4832.260     53.930     82.190  off  
           30    2021.04   weekday_0   2624.784     53.480     82.480  off  
           30    2021.04   weekday_8  13680.000     53.480     82.480  off  
           30    2021.04  weekday_17   7762.174     53.480     82.480  off  

[46728 rows x 12 columns]

In [59]:
daily['off_price_mean']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['peak_price_mean']=dam.loc[dam['cat']=='peak'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['peak_hour_count']=dam.loc[dam['cat']=='peak'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily

volume  price_weight_av  price_max  price_min  price_mean  \
year month day                                                              
2016 1     1    7873.8        42.785251      60.08      35.09   45.982500   
           2    7319.5        47.268532     118.49      40.03   54.114167   
           3    7198.7        48.857812     118.47      44.06   54.959167   
           4    4457.0        50.693024     177.66      46.70   80.164583   
           5    3750.4        47.746609     181.40      42.06   84.503333   
...                ...              ...        ...        ...         ...   
2021 4     26   2092.9        88.683281     299.33      39.94  102.215000   
           27   2048.2        82.150912     299.09      39.96   94.468333   
           28   2130.2        83.346161     299.10      39.90  100.343333   
           29   2134.0        83.738512     296.82      39.79   98.742500   
           30   2198.9        85.068659     295.98      38.97   98.218750   

                boundary1  boundary2  off_price_mean  standard_price_mean  \
year month day                                                              
2016 1     1       42.585     47.595       45.982500                  NaN   
           2       47.055     53.565       54.114167                  NaN   
           3       47.235     52.825       54.959167                  NaN   
           4       63.430     96.610       80.164583                  NaN   
           5       61.095     99.005       84.503333                  NaN   
...                   ...        ...             ...                  ...   
2021 4     26      55.000     85.100      102.215000                  NaN   
           27      54.990     85.050       94.468333                  NaN   
           28      54.970     85.050      100.343333                  NaN   
           29      53.930     82.190       98.742500                  NaN   
           30      53.480     82.480       98.218750                  NaN   

                peak_price_mean  off_hour_count  standard_hour_count  \
year month day                                                         
2016 1     1                NaN              24                  NaN   
           2                NaN              24                  NaN   
           3                NaN              24                  NaN   
           4                NaN              24                  NaN   
           5                NaN              24                  NaN   
...                         ...             ...                  ...   
2021 4     26               NaN              24                  NaN   
           27               NaN              24                  NaN   
           28               NaN              24                  NaN   
           29               NaN              24                  NaN   
           30               NaN              24                  NaN   

                peak_hour_count  center   base  
year month day                                  
2016 1     1                NaN   45.09  40.08  
           2                NaN   50.31  43.80  
           3                NaN   50.03  44.44  
           4                NaN   80.02  46.84  
           5                NaN   80.05  42.14  
...                         ...     ...    ...  
2021 4     26               NaN   70.05  39.95  
           27               NaN   70.02  39.96  
           28               NaN   70.01  39.93  
           29               NaN   68.06  39.80  
           30               NaN   67.98  38.98  

[1947 rows x 15 columns]

In [62]:
daily.loc[2019,7,17]

volume                 6038.500000
price_weight_av          98.235278
price_max               228.750000
price_min                54.010000
price_mean              116.528333
boundary1                82.065000
boundary2               138.155000
off_price_mean           54.968750
standard_price_mean     112.212727
peak_price_mean         224.518000
off_hour_count            8.000000
standard_hour_count      11.000000
peak_hour_count           5.000000
center                  110.110000
base                     54.020000
Name: (2019, 7, 17), dtype: float64

In [63]:
daily.to_csv('dam_daily.csv')